# Section 1 - Data Aquisition

In [149]:
# Import needed libraries
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import io
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

First we will need to get the list of financial centers and their coordinates. We will get the list of centers from the wikipedia page: https://en.wikipedia.org/wiki/Global_Financial_Centres_Index

In [34]:
# Load article, turn into soup and get the <table>s.
website_url = requests.get('https://en.wikipedia.org/wiki/Global_Financial_Centres_Index').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find_all('table', class_='multicol')

In [169]:
# read the table into a list
df=pd.read_html(str(table))
# convert list to dataframe
df=pd.DataFrame(df[0])
# clean first rows of dataframe
df.drop(df.index[0], inplace=True)

# set the dataframe headers
new_header = df.iloc[0] 
df = df[1:] 
df.columns = new_header 

# drop rows and columns we don't want
df.drop(['Change', 'Rating', 'Change'], axis='columns', inplace=True)
df.drop( df[ df['Rank'] == 'Rank' ].index , inplace=True)
df['Rank'] = pd.to_numeric(df['Rank'])
df.drop(df[df['Rank'] > 50].index, inplace=True)
df.set_index('Rank')
# replac names of Washington DC and New York City so that they retrieve the correct coordinates
df.replace('Washington, D.C.', 'Washington', inplace=True)
df.replace('New York City', 'New York', inplace=True)
df

1,Rank,Centre
2,1,New York
3,2,London
4,3,Shanghai
5,4,Tokyo
6,5,Hong Kong
7,6,Singapore
8,7,Beijing
9,8,San Francisco
10,9,Shenzhen
11,10,Zurich


In [170]:
# Get lat, long coordinates for each city
lats = []
longs = []

for city in df['Centre']:
    geolocator = Nominatim(user_agent="my_user_agent")
    loc = geolocator.geocode(city)
    latitude = loc.latitude
    lats.append(latitude)
    longitude = loc.longitude
    longs.append(longitude)

In [171]:
df['Latitude'] = lats
df['Longitude'] = longs

In [172]:
worldMap = folium.Map(zoom_start=10)

# add markers to map
for lat, lng, City in zip(df['Latitude'], df['Longitude'], df['Centre']):
    label = '{}'.format(City)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(worldMap) 

worldMap